In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
NAME = "us_0-74k"

## Make Companies CSV

In [6]:
cols = ['CompanyName', 'twitterUrl', 'linkedInUrl', 'facebookUrl', 'websiteUrl', \
        'city', 'country', 'countryIso', 'state', 'locationString', 'isPrimary', \
        'employeeSizeRange', 'description']
company_df_dict = {colname: [] for colname in cols}
count_lines = 0

for filepath in ["../data/orgCharts_US_0-16k.json", "../data/orgCharts_US_16-37k.json", "../data/orgCharts_US_37-45k.json", "../data/orgCharts_US_45-68k.json", "../data/orgCharts_US_68-74k.json"]: 
    f = open(filepath)
    data = json.load(f)
    for line in data['orgCharts']:
        count_lines += 1
        company_df_dict['CompanyName'].append(line['CompanyName'])

        company_df_dict['twitterUrl'].append(line['stats']['social']['twitterUrl'])
        company_df_dict['linkedInUrl'].append(line['stats']['social']['linkedInUrl'])
        company_df_dict['facebookUrl'].append(line['stats']['social']['facebookUrl'])
        company_df_dict['websiteUrl'].append(line['stats']['social']['websiteUrl'])

        company_df_dict['city'].append(line['stats']['location']['city'])
        company_df_dict['country'].append(line['stats']['location']['country'])
        company_df_dict['countryIso'].append(line['stats']['location']['countryIso'])
        company_df_dict['state'].append(line['stats']['location']['state'])
        company_df_dict['locationString'].append(line['stats']['location']['locationString'])
        company_df_dict['isPrimary'].append(line['stats']['location']['isPrimary'])

        company_df_dict['employeeSizeRange'].append(line['stats']['employeeRange'])

        company_df_dict['description'].append(line['stats']['description'])
    f.close()
    print(count_lines)

print(count_lines)
company_df = pd.DataFrame.from_dict(company_df_dict)

17593
38260
46500
71311
81194
81194


In [7]:
company_df

,CompanyName,twitterUrl,linkedInUrl,facebookUrl,websiteUrl,city,country,countryIso,state,locationString,isPrimary,employeeSizeRange,description
0,Oltmans Construction Co.,https://www.twitter.com/oltmansconstco,https://www.linkedin.com/company/871266,https://www.facebook.com/oltmansconstruction,http://oltmans.com,Whittier,United States,US,None,"Whittier, California, United States",True,200-500,Oltmans Construction Co. is an industrial and ...
1,Humanetics,https://twitter.com/Fibercore1,http://www.linkedin.com/company/fibercore-limited,None,https://www.humaneticsgroup.com/,Las Vegas,United States,US,NV,None,True,200-500,Humanetics is a technology organization with a...
2,Together Labs,None,https://www.linkedin.com/company/togetherlabs/,None,https://togetherlabs.com/,Redwood City,United States,US,None,"Redwood City, California, United States",True,200-500,Together Labs innovates technologies that empo...
3,The Conair Group,https://www.twitter.com/conairgroup,https://www.linkedin.com/company/the-conair-gr...,https://www.facebook.com/ConairGroup/,http://conairgroup.com,Cranberry Twp,United States,US,None,"Cranberry Twp, Pennsylvania, United States",True,200-500,The Conair Group is a plastics company that pr...
4,FortunaPIX,None,None,None,http://www.fortunapix.com,Nashville,United States,US,None,"Nashville, Tennessee, United States",True,200-500,Ed Tech
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81189,Puget Systems,https://twitter.com/pugetsystems,https://www.linkedin.com/company/puget-systems,https://www.facebook.com/PugetSystems,https://www.pugetsystems.com,Auburn,United States,US,None,"Auburn, Washington, United States",True,10-50,America's Custom Computer Leader
81190,RF Arrays,http://twitter.com/RFArrays,http://www.linkedin.com/company/r-f-arrays-sys...,http://www.facebook.com/RFArraysSystems,http://www.rfarrays.com,Portland,United States,US,None,"Portland, Oregon, United States",True,10-50,"RF Arrays, a fabless semiconductor company, de..."
81191,Savage IO,https://www.twitter.com/savageiony,http://www.linkedin.com/company/savage-io,http://www.facebook.com/SavageIO,http://www.savageio.com,Batavia,United States,US,None,"Batavia, New York, United States",True,10-50,Savage IO is a big data storage center that de...
81192,Silver Bullet Technology,None,None,http://www.facebook.com/pages/Silver-Bullet-Te...,http://www.sbullet.com/,Pensacola,United States,US,None,"Pensacola, Florida, United States",True,10-50,Silver Bullet Technology software and solution...


In [8]:
company_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81194 entries, 0 to 81193
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CompanyName        81194 non-null  object
 1   twitterUrl         63318 non-null  object
 2   linkedInUrl        75389 non-null  object
 3   facebookUrl        60688 non-null  object
 4   websiteUrl         81184 non-null  object
 5   city               81191 non-null  object
 6   country            81194 non-null  object
 7   countryIso         81194 non-null  object
 8   state              28297 non-null  object
 9   locationString     62544 non-null  object
 10  isPrimary          81194 non-null  bool  
 11  employeeSizeRange  81194 non-null  object
 12  description        81030 non-null  object
dtypes: bool(1), object(12)
memory usage: 7.5+ MB


In [9]:
company_df['employeeSizeRange'].value_counts()

10-50      48343
50-200     22662
200-500    10189
Name: employeeSizeRange, dtype: int64

### Clean locations - lots of missing states

In [10]:
# Mostly missing states, but they appear in the locationString -- (only "NV" appears), so try to fill in from locationString
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

def fill_state_abbrev(desc):
    if desc is not None: 
        states = us_state_to_abbrev.keys()
        for state in states: 
            if state in desc: 
                abbrev = us_state_to_abbrev[state]
                return abbrev
    return None

def map_to_us_state(x): 
    try: 
        return abbrev_to_us_state[x]
    except:
        return ','


In [11]:
company_df['stateFilled'] = company_df['locationString'].apply(lambda x: fill_state_abbrev(x))
company_df['stateAbbrev'] = company_df.apply(lambda row: row["state"] if row["state"] is not None else row["stateFilled"], axis=1)
company_df['stateName'] = company_df['stateAbbrev'].apply(lambda x: map_to_us_state(x))
company_df.drop(columns=['stateFilled', 'state'], inplace=True)
company_df.rename(columns={'stateName': 'state'}, inplace=True)
company_df[['city', 'state', 'country']] = company_df[['city', 'state', 'country']].fillna('')
# Fill in based on cleaned city/state/country columns, some location strings missing
company_df['locationString'] = company_df.apply(lambda row: row["city"] + ", " + row["state"] + ", " + row["country"], axis=1)

In [12]:
company_df[['city', 'state', 'country', 'locationString']]

,city,state,country,locationString
0,Whittier,California,United States,"Whittier, California, United States"
1,Las Vegas,Nevada,United States,"Las Vegas, Nevada, United States"
2,Redwood City,California,United States,"Redwood City, California, United States"
3,Cranberry Twp,Pennsylvania,United States,"Cranberry Twp, Pennsylvania, United States"
4,Nashville,Tennessee,United States,"Nashville, Tennessee, United States"
...,...,...,...,...
81189,Auburn,Washington,United States,"Auburn, Washington, United States"
81190,Portland,Oregon,United States,"Portland, Oregon, United States"
81191,Batavia,New York,United States,"Batavia, New York, United States"
81192,Pensacola,Florida,United States,"Pensacola, Florida, United States"


### Check # Companies

In [13]:
company_df["CompanyName"].value_counts()

Purely Cloud                              50
Collaboration of Integrated Healthcare    25
Lokalise                                  23
StartX                                    18
MindWeaver                                15
                                          ..
ESI Healthcare Business Solutions          1
Axiom Medical                              1
Hifinite Health                            1
Health & Home Services                     1
M and M Heating and Air Conditioning       1
Name: CompanyName, Length: 38588, dtype: int64

In [14]:
company_df.columns

Index(['CompanyName', 'twitterUrl', 'linkedInUrl', 'facebookUrl', 'websiteUrl',
       'city', 'country', 'countryIso', 'locationString', 'isPrimary',
       'employeeSizeRange', 'description', 'stateAbbrev', 'state'],
      dtype='object')

### Drop unnecessary columns

In [15]:
company_df.drop(columns=['countryIso', 'stateAbbrev', 'isPrimary'], inplace=True)

In [16]:
len(company_df["CompanyName"].value_counts())

38588

### Drop duplicate companies & save companies_df

In [17]:
company_df.drop_duplicates(subset="CompanyName", inplace=True)

# Save
company_df.to_csv(f"companies_{NAME}.csv")

## Make Org Chart CSV

In [91]:
cols = ['CompanyName', 'id', 'title', 'leafMember', 'containingNodeId', 'order', 'parentId', 'section', 'type']
orgchart_df_dict = {colname: [] for colname in cols}
count_lines = 0

for filepath in ["data/orgCharts_US_0-16k.json", "data/orgCharts_US_16-37k.json", "data/orgCharts_US_37-45k.json", "data/orgCharts_US_45-68k.json", "data/orgCharts_US_68-74k.json"]: 
    f = open(filepath)
    data = json.load(f)
    for line in data['orgCharts']:
        count_lines += 1
        employeeNodes = line['employeeNodes']
        for i in range(len(employeeNodes)):
            if line['employeeNodes'][i]['__typename'] == 'OrgChartStructureNode': 

                for col in cols: 
                    if col == 'CompanyName': 
                        orgchart_df_dict['CompanyName'].append(line['CompanyName'])
                    else: 
                        orgchart_df_dict[col].append(line['employeeNodes'][i][col])
            # Print type if not OrgChartStructureNode 
            else: 
                print(line['employeeNodes'][i]['__typename'])
    print(count_lines)
            
orgchart_df = pd.DataFrame.from_dict(orgchart_df_dict)
orgchart_df

17593
38260
46500
71311
81194


,CompanyName,id,title,leafMember,containingNodeId,order,parentId,section,type
0,Oltmans Construction Co.,p-1517109,John Gormly,"{'__typename': 'FlatPosition', 'id': 1517109, ...",None,0,None,orgChart,leaf
1,Oltmans Construction Co.,p-1517136,Charles Roy,"{'__typename': 'FlatPosition', 'id': 1517136, ...",None,1,p-1517109,orgChart,leaf
2,Oltmans Construction Co.,p-1517160,Jason Kakimoto,"{'__typename': 'FlatPosition', 'id': 1517160, ...",None,0,p-1517109,orgChart,leaf
3,Oltmans Construction Co.,p-1517254,John Dang,"{'__typename': 'FlatPosition', 'id': 1517254, ...",None,0,p-1517139,orgChart,leaf
4,Oltmans Construction Co.,p-1517248,Louise Le,"{'__typename': 'FlatPosition', 'id': 1517248, ...",None,17,p-1517109,orgChart,leaf
...,...,...,...,...,...,...,...,...,...
1230687,Puget Systems,p-1974175,Jon Bach,"{'id': 1974175, 'slug': 'jon-bach', 'fullName'...",None,0,None,orgChart,leaf
1230688,RF Arrays,p-1569086,Chandra Deshpandey,"{'id': 1569086, 'slug': 'chandra-deshpandey', ...",None,0,None,orgChart,leaf
1230689,Savage IO,p-1278299,Stephen K. Garvin,"{'id': 1278299, 'slug': 'stephen-k-garvin', 'f...",None,0,None,orgChart,leaf
1230690,Silver Bullet Technology,p-1515338,Bryan Clark,"{'id': 1515338, 'slug': 'bryan-clark', 'fullNa...",None,0,None,orgChart,leaf


### Clean IDs, names

In [92]:
# Clean id's to not include "p-"
orgchart_df["id"] = orgchart_df["id"].apply(lambda x: None if x is None else int(x[2:]) if x[0:2] == 'p-' else x)
orgchart_df["parentId"] = orgchart_df["parentId"].apply(lambda x: None if x is None else int(x[2:]) if x[0:2] == 'p-' else x)

# Rename title to name
orgchart_df.rename(columns={"id": "employeeId", "title": "employeeName"}, inplace=True)

In [93]:
orgchart_df

,CompanyName,employeeId,employeeName,leafMember,containingNodeId,order,parentId,section,type
0,Oltmans Construction Co.,1517109,John Gormly,"{'__typename': 'FlatPosition', 'id': 1517109, ...",None,0,None,orgChart,leaf
1,Oltmans Construction Co.,1517136,Charles Roy,"{'__typename': 'FlatPosition', 'id': 1517136, ...",None,1,1517109,orgChart,leaf
2,Oltmans Construction Co.,1517160,Jason Kakimoto,"{'__typename': 'FlatPosition', 'id': 1517160, ...",None,0,1517109,orgChart,leaf
3,Oltmans Construction Co.,1517254,John Dang,"{'__typename': 'FlatPosition', 'id': 1517254, ...",None,0,1517139,orgChart,leaf
4,Oltmans Construction Co.,1517248,Louise Le,"{'__typename': 'FlatPosition', 'id': 1517248, ...",None,17,1517109,orgChart,leaf
...,...,...,...,...,...,...,...,...,...
1230687,Puget Systems,1974175,Jon Bach,"{'id': 1974175, 'slug': 'jon-bach', 'fullName'...",None,0,None,orgChart,leaf
1230688,RF Arrays,1569086,Chandra Deshpandey,"{'id': 1569086, 'slug': 'chandra-deshpandey', ...",None,0,None,orgChart,leaf
1230689,Savage IO,1278299,Stephen K. Garvin,"{'id': 1278299, 'slug': 'stephen-k-garvin', 'f...",None,0,None,orgChart,leaf
1230690,Silver Bullet Technology,1515338,Bryan Clark,"{'id': 1515338, 'slug': 'bryan-clark', 'fullNa...",None,0,None,orgChart,leaf


## Parse LeafMembers

In [94]:
orgchart_df.iloc[336068]["leafMember"]

{'id': 1578048,
 'slug': 'kris-grindstaff',
 'fullName': 'Kris Grindstaff',
 'role': 'Industrial Networking & Cybersecurity Practice Lead',
 'roleFunction': None,
 'roleAutoFunction': 'eng',
 'description': 'Kris Grindstaff joined Vertech in July of 2011, and he holds the position of Renewable Energy Specialist for Vertech. Kris has extensive experience and skills in designing, programming, testing, and servicing complete automation solutions. Kris’ experience includes everything from servo and motion control drives, PLC’s, HMI’s, to plant-wide SCADA and utility-scale solar plant monitoring and control systems. In addition to his controls programming and SCADA skills, Kris’ skill set includes proficiency in Information Technology areas including database architecture, historian management, site management, server administration, cybersecurity, and network configuration. Kris is typically responsible for projects for all duties covering design, technical expertise, contractual issues, o

In [95]:
cols = ["slug", "role", "roleFunction", "roleAutoFunction", "description", "parentPositionId", "isAdviser", "group", 
       "companyStartDate", "roleStartDate", "location", "invitedAt", "remote", "lastUpdate", "pronoun", "claimedBy"]
for colname in cols:
    orgchart_df[colname] = orgchart_df["leafMember"].apply(lambda leaf_dict: leaf_dict[colname] if leaf_dict is not None and colname in leaf_dict.keys() else None)
    
social_cols = ["twitterUrl", "linkedInUrl", "facebookUrl", "websiteUrl"]
for colname in social_cols:
    orgchart_df[colname] = orgchart_df["leafMember"].apply(lambda leaf_dict: leaf_dict['social'][colname] if leaf_dict is not None and colname in leaf_dict.keys() else None)

cols_to_drop = ['leafMember', 'containingNodeId', 'roleFunction', 'group', 'companyStartDate', 'roleStartDate', 
               'location', 'invitedAt', 'pronoun', 'claimedBy', 'section'] + social_cols
orgchart_df.drop(columns=cols_to_drop, inplace=True)

In [96]:
orgchart_df

,CompanyName,employeeId,employeeName,order,parentId,type,slug,role,roleAutoFunction,description,parentPositionId,isAdviser,remote,lastUpdate
0,Oltmans Construction Co.,1517109,John Gormly,0,None,leaf,john-gormly,Chairman & CEO,admn,With Oltmans Since 1975 California State Unive...,NaN,False,False,2022-07-11T13:27:41.756
1,Oltmans Construction Co.,1517136,Charles Roy,1,1517109,leaf,charles-roy,President,admn,With Oltmans Since 1984 University of Californ...,1517109.0,False,False,2022-07-11T13:30:04.569
2,Oltmans Construction Co.,1517160,Jason Kakimoto,0,1517109,leaf,jason-kakimoto,Vice President & CFO,finc,Jason Kakimoto was named vice president and me...,1517109.0,False,False,2022-07-11T13:42:56.943
3,Oltmans Construction Co.,1517254,John Dang,0,1517139,leaf,john-dang,"Associate Director, Business Development",bd,As a leader in Oltmans’ client relations and b...,1517139.0,False,False,2022-07-11T13:41:20.440
4,Oltmans Construction Co.,1517248,Louise Le,17,1517109,leaf,louise-le,"Director, Marketing",mktg,Joining the team as Oltmans Construction Co.’s...,1517109.0,False,False,2022-07-11T13:40:37.563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230687,Puget Systems,1974175,Jon Bach,0,None,leaf,jon-bach,President,None,Jon founded Puget Systems in December 2000. To...,NaN,False,False,2022-11-09T21:07:54.650
1230688,RF Arrays,1569086,Chandra Deshpandey,0,None,leaf,chandra-deshpandey,CEO & MD,admn,Dr. Chandra Deshpandey is a founding member of...,NaN,False,False,2022-07-21T16:40:42.776
1230689,Savage IO,1278299,Stephen K. Garvin,0,None,leaf,stephen-k-garvin,CEO & Board Member,admn,Steve is a Senior Advisor with Sandhill Capita...,NaN,False,False,2022-08-29T17:17:52.386
1230690,Silver Bullet Technology,1515338,Bryan Clark,0,None,leaf,bryan-clark,CEO,admn,Bryan Clark is the chief executive officer and...,NaN,False,False,2022-07-08T21:28:59.775


### Create Employee table

In [97]:
employee_df = orgchart_df[['employeeId', 'slug', 'CompanyName', 'employeeName', 'order', 'role', 
                           'roleAutoFunction', 'description', 'remote']]
employee_df.set_index('employeeId', inplace=True)

In [99]:
employee_df = employee_df.drop_duplicates()
employee_df

,slug,CompanyName,employeeName,order,role,roleAutoFunction,description,remote
employeeId,,,,,,,,
1517109,john-gormly,Oltmans Construction Co.,John Gormly,0,Chairman & CEO,admn,With Oltmans Since 1975 California State Unive...,False
1517136,charles-roy,Oltmans Construction Co.,Charles Roy,1,President,admn,With Oltmans Since 1984 University of Californ...,False
1517160,jason-kakimoto,Oltmans Construction Co.,Jason Kakimoto,0,Vice President & CFO,finc,Jason Kakimoto was named vice president and me...,False
1517254,john-dang,Oltmans Construction Co.,John Dang,0,"Associate Director, Business Development",bd,As a leader in Oltmans’ client relations and b...,False
1517248,louise-le,Oltmans Construction Co.,Louise Le,17,"Director, Marketing",mktg,Joining the team as Oltmans Construction Co.’s...,False
...,...,...,...,...,...,...,...,...
353480,marin-soljacic,Lightelligence,Marin Soljacic,2,"Co-Founder, Advisor",cnsl,Marin is a professor of Physics at MIT and a c...,False
353477,maurice-steinman,Lightelligence,Maurice Steinman,1,VP of Engineering,eng,Mo has enjoyed a career in the tech industry l...,False
353476,huaiyu-meng,Lightelligence,Huaiyu Meng,0,"Co-Founder, CTO",it,Huaiyu Meng is a co-founder and CTO at [Lighte...,False


In [100]:
employee_df.to_csv(f"employee_{NAME}.csv")

### Create orgchart table

In [101]:
chart_df = orgchart_df[['employeeId', 'parentId']]
chart_df = chart_df.dropna().drop_duplicates()
chart_df

,employeeId,parentId
1,1517136,1517109
2,1517160,1517109
3,1517254,1517139
4,1517248,1517109
5,1517247,1517109
...,...,...
1230335,1299003,1298996
1230336,1299013,1298996
1230337,1299000,1298996
1230371,353477,353475


In [102]:
chart_df.to_csv(f"orgchart_{NAME}.csv")

# Union with international files

In [107]:
companies_intl = pd.read_csv("cleaned_data/companies_intl.csv")
employees_intl = pd.read_csv("cleaned_data/employees_intl.csv")
orgchart_intl = pd.read_csv("cleaned_data/orgchart_intl.csv")

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.
